# 使用Firecrawl

In [ ]:
import os
os.environ["http_proxy"]="127.0.0.1:7890"
os.environ["https_proxy"]="127.0.0.1:7890"

In [ ]:
from openai import OpenAI

# 官方API
# client = OpenAI()

# 第三方低价API
api_key = ""
api_base = ""
client = OpenAI(api_key=api_key, base_url=api_base)

In [ ]:
import telebot

BOT_TOKEN = ""
bot = telebot.TeleBot(BOT_TOKEN)

In [ ]:
@bot.message_handler(commands=['query_news'])
def 实时查询(message):
    pass

In [ ]:
from firecrawl import FirecrawlApp

app = FirecrawlApp(api_key="fc-YOUR_API_KEY")

# Crawl a website:
crawl_status = app.crawl_url(
  'https://firecrawl.dev', 
  params={
    'limit': 100, 
    'scrapeOptions': {'formats': ['markdown', 'html']}
  },
  poll_interval=30
)
print(crawl_status)

https://so.news.cn/#search/0/%E4%B9%A0%E8%BF%91%E5%B9%B3/1/0

# 调用 Perplexity API

In [1]:
import os
os.environ["http_proxy"]="127.0.0.1:7890"
os.environ["https_proxy"]="127.0.0.1:7890"

In [2]:
from openai import OpenAI

# 官方API
# client = OpenAI()

# 第三方低价API
api_key = ""
api_base = ""
client = OpenAI(api_key=api_key, base_url=api_base)

In [3]:
import telebot

BOT_TOKEN = ""
bot = telebot.TeleBot(BOT_TOKEN)

In [4]:
@bot.message_handler(commands=['news'])
def initiate_query_news(message):
    text = "您需要查询什么最新的新闻？"
    sent_msg = bot.send_message(message.chat.id, text, parse_mode="Markdown")
    bot.register_next_step_handler(sent_msg, process_news_query)  # 修改函数名称

In [5]:
import requests
import json

def query_perplexity(user_request):
    url = "https://api.perplexity.ai/chat/completions"
    
    # 在用户请求后追加一句话请求返回来源
    user_request_with_source = user_request + " Please include the source URL of the news."
    
    payload = {
        "model": "llama-3.1-sonar-small-128k-online",
        "messages": [
            {
                "role": "system",
                "content": "Provide a concise and factual response, with clear and precise information."
            },
            {
                "role": "user",
                "content": user_request_with_source  # 动态使用用户请求，并请求来源
            }
        ],
        "max_tokens": 4000,
        "temperature": 0.2,
        "top_p": 0.9,
        "return_citations": True,  # 启用返回引用
        "search_domain_filter": ["perplexity.ai"],
        "return_images": False,
        "return_related_questions": False,
        "search_recency_filter": "month",
        "top_k": 0,
        "stream": False,
        "presence_penalty": 0,
        "frequency_penalty": 1
    }
    
    headers = {
        "Authorization": "Bearer <token>",  # 替换为你的token
        "Content-Type": "application/json"
    }
    
    response = requests.request("POST", url, json=payload, headers=headers)
    
    # 检查响应状态
    if response.status_code == 200:
        data = json.loads(response.text)
        content = data['choices'][0]['message']['content']
        return content  # 返回API的回答
    else:
        return f"Error: {response.status_code} - {response.text}"  # 返回错误信息

In [8]:
# test query_perplexity
# user_request = "what is the latest Xi Jinping News?"
# answer = query_perplexity(user_request)
# print(answer)

In [ ]:
# 修改函数名称，并处理输入请求
def process_news_query(message): 
    user_request = message.text
    perplexity_answer = query_perplexity(user_request)
    bot.send_message(message.chat.id, perplexity_answer, parse_mode="Markdown")
    end_message = "本次查询结束。如想重新查询，请输入“/query_news”。"
    bot.send_message(message.chat.id, end_message, parse_mode=None)

### 似乎还是无法返回新闻的来源，但问题不是很大

In [ ]:
# bot.infinity_polling()
bot.polling()